In [1]:
import numpy as np
import random
import nltk
import json
import pickle
from nltk.stem.lancaster import LancasterStemmer
import tflearn
import tensorflow as tf
from tensorflow.python.framework import ops
stemmer = LancasterStemmer()

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
with open("intents.json") as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']

# loop through sentence in our intents patterns
# tokenize each word, add to words list, add to documents in corpus and add to classes list
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
# find stem for each word, lower and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [4]:
# create training data and array for output
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    # initialise bag of words
    bag = []
    # list of tokenised_words
    tokenized_words = doc[0]
    # stem and lower each word
    tokenized_words = [stemmer.stem(word.lower()) for word in tokenized_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in tokenized_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag        
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
# shuffle and convert to array    
random.shuffle(training)
Data_type = object
training = np.array(training, dtype = Data_type)

X_train = list(training[:, 0])
y_train = list(training[:, 1])

In [5]:
ops.reset_default_graph()

In [6]:
# build network
net = tflearn.input_data(shape = [None, len(X_train[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(y_train[0]), activation = 'softmax')
net = tflearn.regression(net)

# define model and train
model = tflearn.DNN(net)
model.fit(X_train, y_train, n_epoch = 1000, batch_size = 8, show_metric = True)
model.save('../outputs/model.tflearn')

Training Step: 3999  | total loss: 1.09123 | time: 0.010s
| Adam | epoch: 1000 | loss: 1.09123 - acc: 0.9373 -- iter: 24/27
Training Step: 4000  | total loss: 0.98443 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.98443 - acc: 0.9435 -- iter: 27/27
--
INFO:tensorflow:c:\Users\dell\Moon\Chatbot\outputs\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
# save model
pickle.dump({'words' : words, 'classes' : classes, 'X_train' : X_train, 'y_train' : y_train}, open("training_data", "wb"))

In [8]:
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
X_train = data['X_train']
y_train = data['y_train']

with open('intents.json') as json_data:
    intents = json.load(json_data)

In [9]:
# load model
model.load('../outputs/model.tflearn')

INFO:tensorflow:Restoring parameters from c:\Users\dell\Moon\Chatbot\outputs\model.tflearn


In [10]:
def get_sentences(sentence):
    # tokenise the pattern and stem words
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words


# get bag of words
def bags(sentence, words, show_details = False):
    sentence_words = get_sentences(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i , w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" %w)
                    
    return(np.array(bag))

In [11]:
ERROR_THRESHOLD = 0.25

# generate probabilities from the model and filter out predictions below a threshold
def classify(sentence):
    results = model.predict([bags(sentence, words)])[0]
    reults = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    # sort by strength of probsbility
    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID = '123', show_details = False):
    results = classify(sentence)
    # if we have a classification, find a matching intent tag
    if results:
        # loop as long as there are matches
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # get response
                    return print(random.choice(i['responses']))
                
            results.pop(0)

In [12]:
# define structure to hold user content
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # get probabilities from the model
    results = model.predict([bags(sentence, words)])[0]
    # filter out probabilities from the model
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    # sort by probability strength
    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return intent and probability
    return return_list

def response(sentence, userID = '123', show_details = False):
    results = classify(sentence)
    # if there's a classification find the intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching first result
                if i['tag'] == results[0][0]:
                    # set context for intent
                    if 'context_set' in i:
                        if show_details: print('context:', i['context_set'])
                        context[userID] = i['context_set']
                        
                    # check if intent fits the context of user conversation
                    if not 'context_filter' in i or (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print('tag:', i['tag'])
                        return print(random.choice(i['responses']))
    
            results.pop(0)

In [13]:
response('we want to rent a moped')
response('today')
classify('today')

Are you looking to rent today or later this week?
Same-day rentals please call 1-800-MYMOPED


[('today', 0.9171227)]

In [14]:
response("Hi there!", show_details=True)

context: 
tag: greeting
Good to see you again


In [16]:
response("thanks")

My pleasure
